# Multi-Layer Perceptron

Just to get familiar with the tool, we're going to be making something a bit more basic here.

In [2]:
import mlx.core as mlx
import mlx.nn as nn
import mlx.optimizers as optim

import numpy as np

# Initializers

According to what's next here, the model is defined as the MLP class which inherits from `mlx.nn.Module`. We follow the standard practice to make a new model.

1. Define an `__init__` where the parameters and/or submodules are setup.
2. Define a `__call__` where the computationo is implemented.

In [ ]:
class MLP(nn.Module):
    def __init__(
            self,
            num_layers: int,
            input_dim: int,
            hidden_dim: int,
            output_dim: int
    ):
        super().__init__()
        layer_sizes = [input_dim] + [hidden_dim] * num_layers + [output_dim]
        self.layers = [
            nn.Linear(idim, odim)
            for idim, odim in zip(layer_sizes[:-1], layer_sizes[1:])
        ]

    def __call__( self, x ):
        for l in self.layers[:-1]:
            x = mlx.maximum(l(x), 0.0)
        return self.layers[-1](x)